In [1]:
import os
import pandas as pd
from ipynotifyer import notifyOnComplete as nf
import zipfile #i am not giving it a name cause i intend to use it only once

In [2]:
path = '../../Data/ACS/1y2013_All_geography'

In [3]:
def getFiles(folder, format, full=True, filtr=None):
    for f in os.listdir(folder):
        if f.endswith(format) and f.startswith('e'):
            if filtr==None or filtr not in f:
                if full:
                    yield os.path.join(folder, f)
                else:
                    yield f


In [4]:
txts = list(getFiles(path, '.txt'))
txts[:10]

['../../Data/ACS/1y2013_All_geography/e20131ak0001000.txt',
 '../../Data/ACS/1y2013_All_geography/e20131ak0002000.txt',
 '../../Data/ACS/1y2013_All_geography/e20131ak0003000.txt',
 '../../Data/ACS/1y2013_All_geography/e20131ak0004000.txt',
 '../../Data/ACS/1y2013_All_geography/e20131ak0005000.txt',
 '../../Data/ACS/1y2013_All_geography/e20131ak0006000.txt',
 '../../Data/ACS/1y2013_All_geography/e20131ak0007000.txt',
 '../../Data/ACS/1y2013_All_geography/e20131ak0008000.txt',
 '../../Data/ACS/1y2013_All_geography/e20131ak0009000.txt',
 '../../Data/ACS/1y2013_All_geography/e20131ak0010000.txt']

In [5]:
cols = [
'FILEID',
'FILETYPE',
'STUSAB',
'CHARITER',
'SEQUENCE',
'LOGRECNO',
]

In [19]:
pumas = pd.read_csv('../../Data/geo_id/pumas.csv', index_col=0, dtype='str')[[4, 49]]
pum = pumas['4'].tolist()
len(pum)

2378

In [10]:
def getTable(fname, gf=None):
    df = pd.read_csv(fname, 
                             dtype={5:str, 4:str},
                             header=None, low_memory=False,
                             error_bad_lines=True)
    seq = df.iloc[1,4]
    st = fname[-13:-11]

    df.columns = cols + ['%s_%d' % (seq , i+1) for i in xrange(df.shape[1] - 6)]
    df['st'] = st
    
    if gf:
        return df[df['LOGRECNO'].isin(gf)].iloc[:,5:].set_index('LOGRECNO')
    else:
        return df.iloc[:,5:].set_index('LOGRECNO')


In [11]:
t1 = getTable(txts[10], gf=pum)

In [12]:
t1.shape

(17, 140)

In [13]:
@nf
def aggTables(folder, gf=None):    
    
    byState ={}
    for t in getFiles(folder, '.txt'):
        st = t[-13:-11]
        
        x = byState.get(st,[])
        x.append(getTable(t, gf=gf))
        byState[st] = x
        
    # concat horisontally inside the state
    for k, v in byState.iteritems() :
        byState[k] = pd.concat(v,1)
        
    # concat vertically all the states
    return pd.concat(byState.values())

In [14]:
# get all data for all PUMAs
# mdf = aggTables(path, gf=pum)

mdf2 = aggTables(path, gf=pum)

In [15]:
mdf2.shape

(6340, 32922)

In [16]:
mdf2.head(5)

,0001_1,0001_2,st,0002_1,0002_2,0002_3,0002_4,0002_5,0002_6,0002_7,...,0170_66,0170_67,0170_68,0170_69,0170_70,0170_71,0170_72,0170_73,0170_74,st
LOGRECNO,,,,,,,,,,,,,,,,,,,,,
0000010,3776,1706,va,213621,105709,5099,5915,6072,3937,3300,...,86541,4793,85609,90402,3187,3187,3187,15003,75156,va
0000011,13614,6347,va,780918,389902,19998,20818,21492,13636,11849,...,299627,12020,297642,309662,9637,9713,9713,47529,260611,va
0000012,18432,8544,va,1057565,525408,27471,29015,30288,18158,16456,...,410632,18171,407639,425810,13766,13842,13842,66687,357358,va
0000013,1769,618,va,103000,49272,2843,3213,3173,1886,3179,...,37917,1348,37674,39022,1369,1369,1369,8590,30402,va
0000014,3197,1519,va,224906,111593,6278,5070,4389,2327,1521,...,96526,2874,96099,98973,3337,3337,3337,22322,76330,va


In [18]:
# save to csv
mdf2.to_csv('../../Data/raw_PUMAS_1yUS.csv')

In [ ]:
# save column names as a csv
pd.DataFrame(mdf.columns).to_csv('../../Data/PUMAS_series.csv')